In [1]:
import os
import torch
from transformers import pipeline

/home/andresmtr/miniconda3/envs/AndTextG3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ruta donde están los archivos .txt
ruta_entrada = "text/en/final"
# Ruta donde se guardarán los resúmenes
ruta_salida = "text/en/union"

In [3]:
# Crear la ruta de salida si no existe
os.makedirs(ruta_salida, exist_ok=True)

In [4]:
def unir_txts(ruta_entrada, ruta_salida, nombre_salida="resultado.txt"):
    # Crear la ruta completa del archivo de salida
    ruta_archivo_salida = os.path.join(ruta_salida, nombre_salida)
    
    try:
        # Verificar si la carpeta de entrada existe
        if not os.path.exists(ruta_entrada):
            print(f"La ruta de entrada {ruta_entrada} no existe.")
            return
        
        # Crear la carpeta de salida si no existe
        if not os.path.exists(ruta_salida):
            os.makedirs(ruta_salida)
        
        # Obtener todos los archivos .txt de la carpeta de entrada
        archivos_txt = [f for f in os.listdir(ruta_entrada) if f.endswith(".txt")]
        
        if not archivos_txt:
            print("No se encontraron archivos .txt en la ruta de entrada.")
            return
        
        with open(ruta_archivo_salida, "w", encoding="utf-8") as archivo_salida:
            for i, archivo in enumerate(archivos_txt):
                ruta_archivo = os.path.join(ruta_entrada, archivo)
                with open(ruta_archivo, "r", encoding="utf-8") as f:
                    contenido = f.read()
                
                archivo_salida.write(contenido)
                
                # Agregar un salto de página entre archivos, excepto después del último archivo
                if i < len(archivos_txt) - 1:
                    archivo_salida.write("\n\f\n")
        
        print(f"Archivos combinados con éxito en: {ruta_archivo_salida}")
    
    except Exception as e:
        print(f"Ocurrió un error: {e}")

In [5]:
unir_txts(ruta_entrada, ruta_salida)

Archivos combinados con éxito en: text/en/union/resultado.txt


In [6]:
# Configurar el dispositivo para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
# Cargar el modelo Llama
# Configurar el pipeline para Llama 3.2
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,  # Usar bfloat16 para optimizar uso de memoria
    device_map="auto"  # Mapear automáticamente a la GPU si está disponible
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.96it/s]


In [8]:
def process_text(file_path):
    try:
        # Leer el archivo de texto
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

        # Definir el prompt
        prompt = f"With all text Create title and description, the text is about week news: \n\n{text}\n\n "

        # Generar respuesta con el modelo Llama
        response = pipe(
            prompt,
            max_new_tokens=1024,  # Ajustar según el tamaño del contexto que necesites
            temperature=1.0  # Puedes ajustar la temperatura para controlar la creatividad
        )

        # Obtener el texto generado
        generated_text = response[0]["generated_text"]

        generated_text = generated_text.replace(text, '')

        generated_text = generated_text.replace('Create title and description about the next text:', '')

        # Guardar el texto en un archivo
        with open("text/en/union/descri_text_en.txt", "w", encoding="utf-8") as f:
            f.write(generated_text)
            
    except FileNotFoundError:
        print(f"El archivo {file_path} no existe.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")

In [9]:
process_text(file_path='text/en/union/resultado.txt')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [10]:
# Liberar memoria GPU
torch.cuda.empty_cache()